# Microsoft SharePoint

> [Microsoft SharePoint](https://en.wikipedia.org/wiki/SharePoint)是由微软开发的一个基于网站的协作系统，它使用工作流应用程序、“列表”数据库以及其他 Web 部件和安全功能来赋能业务团队协同工作。

本笔记本介绍了如何从 [SharePoint 文档库](https://support.microsoft.com/en-us/office/what-is-a-document-library-3b5976dd-65cf-4c9e-bf5a-713c10ca2872) 加载文档。默认情况下，文档加载器会加载 `pdf`、`doc`、`docx` 和 `txt` 文件。您可以通过提供适当的解析器来加载其他文件类型（下方有更多说明）。

## 先决条件
1. 按照这些 [Microsoft 标识平台](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app) 说明注册一个应用程序。
2. 完成注册后，Azure 门户将显示应用注册的“概述”窗格。您将看到应用程序（客户端）ID。此值也称为“客户端 ID”，它在 Microsoft 标识平台中唯一标识您的应用程序。
3. 在您将要遵循的 **第 1 项** 步骤中，您可以将重定向 URI 设置为 `https://login.microsoftonline.com/common/oauth2/nativeclient`
4. 在您将要遵循的 **第 1 项** 步骤中，在“应用程序机密”部分生成一个新密码（`client_secret`）。
5. 遵循此 [文档](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-configure-app-expose-web-apis#add-a-scope) 中的说明，向您的应用程序添加以下 `SCOPES`（`offline_access` 和 `Sites.Read.All`）。
6. 要从您的**文档库**检索文件，您将需要它的 ID。要获取它，您需要 `租户名称`、`集合 ID` 和 `子站点 ID` 的值。
7. 要查找您的 `租户名称`，请遵循此 [文档](https://learn.microsoft.com/en-us/azure/active-directory-b2c/tenant-management-read-tenant-name) 中的说明。获取后，只需从值中删除 `.onmicrosoft.com` 并保留其余部分作为您的 `租户名称`。
8. 要获取您的 `集合 ID` 和 `子站点 ID`，您需要您的 **SharePoint** `站点名称`。您的 `SharePoint` 站点 URL 格式为 `https://<tenant-name>.sharepoint.com/sites/<site-name>`。此 URL 的最后一部分是 `站点名称`。
9. 要获取站点 `集合 ID`，请在浏览器中访问此 URL：`https://<tenant>.sharepoint.com/sites/<site-name>/_api/site/id`，然后复制 `Edm.Guid` 属性的值。
10. 要获取 `子站点 ID`（或 web ID），请使用：`https://<tenant>.sharepoint.com/sites/<site-name>/_api/web/id`，然后复制 `Edm.Guid` 属性的值。
11. `SharePoint 站点 ID` 的格式为：`<tenant-name>.sharepoint.com,<Collection ID>,<subsite ID>`。您可以保留该值以在下一步中使用。
12. 访问 [Graph Explorer Playground](https://developer.microsoft.com/en-us/graph/graph-explorer) 以获取您的 `文档库 ID`。第一步是确保您已登录到与您的 **SharePoint** 站点关联的帐户。然后，您需要向 `https://graph.microsoft.com/v1.0/sites/<SharePoint site ID>/drive` 发出请求，响应将返回一个包含 `id` 字段的有效负载，该字段保存您的 `文档库 ID`。

## 🧑 从 SharePoint 文档库引入文档的说明

### 🔑 身份验证

默认情况下，`SharePointLoader` 期望 `CLIENT_ID` 和 `CLIENT_SECRET` 的值分别存储在名为 `O365_CLIENT_ID` 和 `O365_CLIENT_SECRET` 的环境变量中。您可以通过在应用程序的根目录中设置 `.env` 文件来传递这些环境变量，或者在脚本中使用以下命令。

```python
os.environ['O365_CLIENT_ID'] = "YOUR CLIENT ID"
os.environ['O365_CLIENT_SECRET'] = "YOUR CLIENT SECRET"
```

此加载器使用一种称为 [*代表用户*](https://learn.microsoft.com/en-us/graph/auth-v2-user?context=graph%2Fapi%2F1.0&view=graph-rest-1.0) 的身份验证。这是一个带有用户同意的 2 步身份验证。当您实例化加载器时，它将调用并打印用户必须访问以授予应用程序所需权限的 URL。然后，用户必须访问此 URL 并授予应用程序同意。之后，用户必须复制生成的页面 URL 并将其粘贴回控制台。该方法随后将返回 True，表示登录尝试成功。

```python
from langchain_community.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID")
```

完成身份验证后，加载器将在 `~/.credentials/` 文件夹中存储一个令牌（`o365_token.txt`）。此令牌可用于稍后进行身份验证，而无需执行前面解释的复制粘贴步骤。要使用此令牌进行身份验证，您需要在实例化加载器时将 `auth_with_token` 参数更改为 True。

```python
from langchain_community.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", auth_with_token=True)
```

### 🗂️ 文档加载器

#### 📑 从文档库目录加载文档

`SharePointLoader` 可以从文档库中的特定文件夹加载文档。例如，您想加载存储在文档库的 `Documents/marketing` 文件夹中的所有文档。

```python
from langchain_community.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", folder_path="Documents/marketing", auth_with_token=True)
documents = loader.load()
```

如果您收到 `Resource not found for the segment` 错误，请尝试使用 `folder_id` 而不是文件夹路径，该 ID 可以从 [Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer) 获取

```python
loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", auth_with_token=True
                          folder_id="<folder-id>")
documents = loader.load()
```

如果您希望从根目录加载文档，可以省略 `folder_id`、`folder_path` 和 `documents_ids`，加载器将加载根目录。
```python
# 从根目录加载文档
loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", auth_with_token=True)
documents = loader.load()
```

结合使用 `recursive=True`，您可以轻松加载整个 SharePoint 中的所有文档：
```python
# 从根目录加载文档
loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID",
                          recursive=True,
                          auth_with_token=True)
documents = loader.load()
```

#### 📑 从文档 ID 列表加载文档

另一种可能性是为每个要加载的文档提供一个 `object_id` 列表。为此，您需要查询 [Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer) 以查找您感兴趣的所有文档 ID。[此链接](https://learn.microsoft.com/en-us/graph/api/resources/onedrive?view=graph-rest-1.0#commonly-accessed-resources) 提供了有助于检索文档 ID 的端点列表。

例如，要检索存储在 `data/finance/` 文件夹中的所有对象的有关信息，您需要向以下地址发出请求：`https://graph.microsoft.com/v1.0/drives/<document-library-id>/root:/data/finance:/children`。一旦您获得了感兴趣的 ID 列表，就可以使用以下参数实例化加载器。

```python
from langchain_community.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", object_ids=["ID_1", "ID_2"], auth_with_token=True)
documents = loader.load()
```

#### 📑 选择支持的文件类型和首选解析器
默认情况下，`SharePointLoader` 加载在 [`document_loaders/parsers/registry`](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/document_loaders/parsers/registry.py#L10-L22) 中定义的文件类型，并使用默认解析器（如下所示）。
```python
def _get_default_parser() -> BaseBlobParser:
    """获取默认的 MIME 类型解析器。"""
    return MimeTypeBasedParser(
        handlers={
            "application/pdf": PyMuPDFParser(),
            "text/plain": TextParser(),
            "application/msword": MsWordParser(),
            "application/vnd.openxmlformats-officedocument.wordprocessingml.document": (
                MsWordParser()
            ),
        },
        fallback_parser=None,
    )
```
您可以将 `handlers` 参数传递给 `SharePointLoader` 来覆盖此行为。
传递一个字典，该字典将文件扩展名（如 `"doc"`、`"pdf"` 等）或 MIME 类型（如 `"application/pdf"`、`"text/plain"` 等）映射到解析器。请注意，您必须仅使用文件扩展名或 MIME 类型，而不能混合使用它们。

文件扩展名不要包含前导点。

```python
# 使用文件扩展名：
handlers = {
    "doc": MsWordParser(),
    "pdf": PDFMinerParser(),
    "mp3": OpenAIWhisperParser()
}

# 使用 MIME 类型：
handlers = {
    "application/msword": MsWordParser(),
    "application/pdf": PDFMinerParser(),
    "audio/mpeg": OpenAIWhisperParser()
}

loader = SharePointLoader(document_library_id="...",
                            handlers=handlers # 将 handlers 传递给 SharePointLoader
                            )
```
如果多个文件扩展名映射到相同的 MIME 类型，则最后一个字典项将生效。
示例：
```python
# 'jpg' 和 'jpeg' 都映射到 'image/jpeg' MIME 类型。SecondParser() 将用于解析所有 jpg/jpeg 文件。
handlers = {
    "jpg": FirstParser(),
    "jpeg": SecondParser()
}
```